In [1]:
import duckdb
import pandas as pd
import numpy as np
import re
import unidecode
import numpy as np
from rapidfuzz import fuzz
from tqdm import tqdm

def process_name(name, words_to_reverse, slug=True, first_name_initial=None):
    name = name.lower()
    name = unidecode.unidecode(name)
    name = re.sub('-', ' ', name)
    name = re.sub(r'[^a-z\s]', '', name)
    
    words = name.split()
    
    if words_to_reverse > 0:
        if len(words) > words_to_reverse:
            words = words[words_to_reverse:] + words[0:words_to_reverse]
            if first_name_initial:
                words[0] = words[0][0]
        
    if slug:
        name = '-'.join(words)
    else:
        name = ' '.join(words)
        
    return name

In [2]:
con = duckdb.connect("E:/duckdb/tennis.duckdb", read_only=True)
tennis_markets = con.execute("""
SELECT *

FROM competition_mappings c
INNER JOIN market_summaries m
ON c.market_id = m.market_id
""").df()

sofascore_events = con.execute("""
                               SELECT * 
                               FROM sofascore_events 
                               WHERE tournament_category IN ('ATP','WTA','Challenger','ITF Men','ITF Women')
                               AND match_status IN ('Ended','Retired', 'Walkover', 'Defaulted', 'Player 2 defaulted, player 1 won')
                               AND winnerCode IN (1,2)
                               """).df()

match_stats_raw = con.execute("""
    SELECT match_id, key, home, away
    FROM sofascore_match_stats
    WHERE period = 'ALL'
""").df()

con.close()

excluded_selection_names = pd.read_csv('../mappings/excluded_selection_names.csv', header=None)[0].tolist()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
# Preprocessing
# TODO: Functionalise
tennis_markets = tennis_markets[~tennis_markets['selection_name'].str.contains("/")]
tennis_markets = tennis_markets[~tennis_markets['selection_name'].isin(excluded_selection_names)]
tennis_markets['bf_name'] = [process_name(x, 0, False) for x in tennis_markets['selection_name']]
tennis_markets['FORMATTED_DATE'] = pd.to_datetime(tennis_markets['FORMATTED_DATE'])
betfair_names = tennis_markets[['bf_name']].drop_duplicates()
sofascore_events = sofascore_events[~sofascore_events['home_team'].str.contains('/')]
sofascore_events = sofascore_events[~sofascore_events['away_team'].str.contains('/')]
sofascore_events = sofascore_events[sofascore_events['match_status'] != 'Not started']
sofascore_events['event_fetch_date'] = pd.to_datetime(sofascore_events['event_fetch_date'])
sofascore_events['home_clean_name'] = [process_name(x, 1, False) for x in sofascore_events['home_team_slug']]
sofascore_events['away_clean_name'] = [process_name(x, 1, False) for x in sofascore_events['away_team_slug']]
sofascore_events['id'] = sofascore_events['id'].astype(int)
sofascore_events.loc[sofascore_events['match_status'] == 'Player 2 defaulted, player 1 won', 'match_status'] = 'Defaulted'

In [4]:
sofascore_events['home_winner'] = 0
sofascore_events.loc[sofascore_events['winnerCode'] == 1,'home_winner'] = 1
sofascore_events['away_winner'] = 0
sofascore_events.loc[sofascore_events['winnerCode'] == 2,'away_winner'] = 1

In [5]:
# Define the mapping dictionary
category_mapping = {
    'Qualification': ['Qualification', 'Qualification round', 'Qualification round 1', 
                      'Qualification round 2', 'Qualification Final'],
    'Late Stages': ['Final', 'Semifinals', 'Quarterfinals', 'Match for 3rd place'],
    'Middle Stages': ['Round of 32', 'Round of 16', '1/16-finals (R32)', '1/8-finals (R16)'],
    'Early Stages': ['Round of 64', 'Round of 128', '1/32-finals (R64)', '1/64-finals (R128)', 'R128']
}

# Create a flat dictionary for efficient mapping
flat_mapping = {round_name: category 
                for category, rounds in category_mapping.items() 
                for round_name in rounds}

# Apply the categorization efficiently
sofascore_events['tournament_round_category'] = sofascore_events['tournament_round'].map(flat_mapping).fillna('Other')

In [6]:
# These are to be updated to improve coverage / accuracy
player_name_mapping = pd.read_csv('../mappings/player_name_mapping.csv')
market_match_mapping = pd.read_csv('../mappings/market_match_mapping.csv')
market_match_mapping['market_id'] = market_match_mapping['market_id'].astype(str).str.pad(9, fillchar='0', side='right')

In [7]:
sofascore_columns = ['id', 'groundType', 'tournament_name', 'tournament_category', 'tournament_points', 'tournament_round_category', 'home_winner', 'away_winner', 
                     'home_score_period1', 'home_score_period2','home_score_period3', 'home_score_period4','home_score_period5',
                     'away_score_period1', 'away_score_period2','away_score_period3', 'away_score_period4','away_score_period5',
                     'datetime', 'home_clean_name', 'away_clean_name', 'match_status']
sofascore_events_base = sofascore_events[sofascore_columns].copy()

In [8]:
# Define the columns to pivot
home_columns = [col for col in sofascore_events_base.columns if col.startswith('home_')]
away_columns = [col for col in sofascore_events_base.columns if col.startswith('away_')]
common_columns = [col for col in sofascore_events_base.columns if not col.startswith('home_') and not col.startswith('away_')]
home_df = sofascore_events_base[common_columns + home_columns].copy()
home_df['position'] = 'home'
home_df.columns = [col.replace('home_', '') if col.startswith('home_') else col for col in home_df.columns]
away_df = sofascore_events_base[common_columns + away_columns].copy()
away_df['position'] = 'away'
away_df.columns = [col.replace('away_', '') if col.startswith('away_') else col for col in away_df.columns]

In [9]:
sofascore_events_base_unpiv = pd.concat([home_df, away_df], ignore_index=True)
sofascore_events_base_unpiv = sofascore_events_base_unpiv.sort_values(['id', 'position'])
sofascore_events_base_unpiv = sofascore_events_base_unpiv.reset_index(drop=True)
sofascore_events_base_unpiv = sofascore_events_base_unpiv.merge(player_name_mapping, left_on='clean_name', right_on='name', how='left').drop(columns='name')
sofascore_events_base_unpiv.loc[sofascore_events_base_unpiv['index'].isna(),'index'] = -1
sofascore_events_base_unpiv['index'] = sofascore_events_base_unpiv['index'].astype(int)

In [10]:
# sofascore_events_base_unpiv.query('index == -1')['clean_name'].unique()

In [11]:
bf_matched_tennis_markets = tennis_markets.merge(market_match_mapping, on='market_id').query("result != 'REMOVED'")
# Start with your provided line
summed_volumes = bf_matched_tennis_markets.groupby(['market_id','id'])['pp_volume'].sum().reset_index()
# Now, select the rows with maximum pp_volume for each market_id
max_market_vols = summed_volumes.loc[summed_volumes.groupby('id')['pp_volume'].idxmax()]

bf_matched_tennis_markets = bf_matched_tennis_markets.merge(max_market_vols[['market_id','id']], on=['market_id','id'])
bf_matched_tennis_markets = bf_matched_tennis_markets.merge(player_name_mapping, left_on='bf_name', right_on='name').drop(columns='name')
bf_matched_tennis_markets_base = bf_matched_tennis_markets[['index', 'id', 'market_id','result', 'pp_min','pp_max','pp_wap','pp_ltp','pp_volume', 'ip_min','ip_max', 'ip_wap', 'ip_ltp', 'ip_volume']].copy()

In [13]:
# Melt the dataframe to create separate rows for home and away
match_stats = pd.melt(match_stats_raw, id_vars=['match_id', 'key'],
                           value_vars=['home', 'away'],
                           var_name='team', value_name='value')
# Pivot the data
match_stats = match_stats.pivot_table(values='value',
                                                index=['match_id', 'team'],
                                                columns='key', aggfunc='first')

# Reset index to make match_id, period, and team regular columns
match_stats.reset_index(inplace=True)
match_stats['match_id'] = match_stats['match_id'].astype(int)

key,match_id,team,aces,backhandErrors,backhandUnforcedErrors,backhandWinners,breakPointsSaved,breakPointsScored,doubleFaults,dropShotUnforcedErrors,...,secondServeAccuracy,secondServePointsAccuracy,serviceGamesTotal,serviceGamesWon,servicePointsScored,tiebreaks,unforcedErrorsTotal,volleyUnforcedErrors,volleyWinners,winnersTotal
0,10000150,away,4,NaN,NaN,NaN,6/6 (100%),3,5,NaN,...,16/21 (76%),10/21 (48%),9,9,36,0,NaN,NaN,NaN,NaN
1,10000150,home,1,NaN,NaN,NaN,2/5 (40%),0,2,NaN,...,19/21 (90%),13/21 (62%),9,6,29,0,NaN,NaN,NaN,NaN
2,10000152,away,0,NaN,NaN,NaN,1/7 (14%),2,5,NaN,...,20/25 (80%),7/25 (28%),9,3,20,0,NaN,NaN,NaN,NaN
3,10000152,home,2,NaN,NaN,NaN,3/5 (60%),6,1,NaN,...,14/15 (93%),10/15 (67%),8,6,28,0,NaN,NaN,NaN,NaN
4,10000176,away,3,NaN,NaN,NaN,0/0 (0%),4,1,NaN,...,18/19 (95%),14/19 (74%),8,8,32,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313563,9999955,home,1,NaN,NaN,NaN,4/6 (66%),5,0,NaN,...,15/15 (100%),8/15 (53%),9,7,37,0,NaN,NaN,NaN,NaN
1313564,9999956,away,1,NaN,NaN,NaN,6/10 (60%),0,1,NaN,...,9/10 (90%),3/10 (30%),7,3,23,0,NaN,NaN,NaN,NaN
1313565,9999956,home,2,NaN,NaN,NaN,0/0 (0%),4,1,NaN,...,8/9 (89%),7/9 (78%),8,8,33,0,NaN,NaN,NaN,NaN
1313566,9999957,away,2,NaN,NaN,NaN,3/9 (33%),7,6,NaN,...,22/28 (79%),11/28 (39%),14,8,42,0,NaN,NaN,NaN,NaN


In [50]:
base_table = sofascore_events_base_unpiv.merge(bf_matched_tennis_markets_base, on=['index','id'],how='left')
base_table = base_table.merge(match_stats, left_on=['id','position'], right_on=['match_id', 'team'],how='left')
base_table.drop(columns=['match_id','team'],inplace=True)

In [51]:
# Function to safely split the string
def safe_split(x):
    parts = str(x).split('/', 1)
    return parts + [np.nan] * (2 - len(parts))

for col in ['breakPointsSaved','firstReturnPoints','firstServeAccuracy','firstServePointsAccuracy','secondReturnPoints','secondServeAccuracy','secondServePointsAccuracy']:
    # Apply the safe split function
    base_table[[f'{col}', f'{col}Attempted']] = base_table[col].apply(safe_split).tolist()
    
    # Clean up the 'backhandWinners_attempted' column
    base_table[f'{col}Attempted'] = base_table[f'{col}Attempted'].str.split().str[0]
    
    # Convert to numeric, coercing errors to NaN
    base_table[f'{col}'] = pd.to_numeric(base_table[f'{col}'], errors='coerce')
    base_table[f'{col}Attempted'] = pd.to_numeric(base_table[f'{col}Attempted'], errors='coerce')

In [65]:
def calculate_elo(df, k_factor=32, initial_rating=1500):
    # Ensure the dataframe is sorted by datetime and then by id
    df = df.sort_values(['datetime', 'id']).reset_index(drop=True)
    
    # Initialize player ratings dictionary
    player_ratings = {}
    
    # Initialize lists to store ELO history
    pre_match_elo = []
    post_match_elo = []
    
    # Process matches in pairs
    for _, match in tqdm(df.groupby('id')):
        if len(match) != 2:
            raise ValueError(f"Match {match['id'].iloc[0]} does not have exactly 2 players")
        
        player1, player2 = match['index'].values
        winner = match.loc[match['winner'] == 1, 'index'].values[0]
        
        # Get current ratings (or initial rating if first match)
        rating1 = player_ratings.get(player1, initial_rating)
        rating2 = player_ratings.get(player2, initial_rating)
        
        # Store pre-match ratings
        pre_match_elo.extend([rating1, rating2])
        
        # Calculate expected scores
        expected_score1 = 1 / (1 + 10 ** ((rating2 - rating1) / 400))
        expected_score2 = 1 - expected_score1
        
        # Calculate new ratings
        if winner == player1:
            new_rating1 = rating1 + k_factor * (1 - expected_score1)
            new_rating2 = rating2 + k_factor * (0 - expected_score2)
        else:
            new_rating1 = rating1 + k_factor * (0 - expected_score1)
            new_rating2 = rating2 + k_factor * (1 - expected_score2)
        
        # Update player ratings
        player_ratings[player1] = new_rating1
        player_ratings[player2] = new_rating2
        
        # Store post-match ratings
        post_match_elo.extend([new_rating1, new_rating2])
    
    return pre_match_elo, post_match_elo

# If the results look good, you can calculate for the full dataset
base_table['pre_match_elo'], base_table['post_match_elo'] = calculate_elo(base_table)

100%|██████████| 799397/799397 [02:45<00:00, 4839.39it/s]


In [69]:
con = duckdb.connect("E:/duckdb/tennis.duckdb")
con.execute("DROP TABLE base_table")
con.execute("CREATE TABLE base_table AS SELECT * FROM base_table")
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))